# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [22]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tqdm

import cv2
import pyrealsense2 as rs

sns.set_style()

In [5]:
RECORDING_DIR = 'H:\Recordings'
EXERCISES_FILE_NAME = 'Exercises.json'

## Data Loading

Firstly we import all the recordings into the notebook.

In [6]:
recording_jsons = []
for file in os.listdir(RECORDING_DIR):
  if (file.endswith('.json') and 
      not file.endswith('Skeleton.json') and 
      not file.endswith(EXERCISES_FILE_NAME)):
    with open(file=os.path.join(RECORDING_DIR, file), mode='r') as file:
      data = json.load(file)
      recording_jsons.append(data)

len(recording_jsons)

44

Then we load the Exercises from the exercise file.

In [7]:
exercises_json = []

with open(file=os.path.join(RECORDING_DIR, EXERCISES_FILE_NAME), mode='r') as file:
  exercises_json = json.load(file)['Exercises']

len(exercises_json)

13

Next we extract all recordings from the sessions.

In [8]:
recording_paths_rs = []
recording_paths_orbbec = []

for recording in recording_jsons:
  for camera in recording['Cameras']:
    if camera['Type'] == 'Realsense':
      recording_paths_rs.append(camera['FileName'])

len(recording_paths_rs)

44

In [27]:
def load_recording_rs(recording_path_rs, num_frames: int):
  pipeline = rs.pipeline()
  config = rs.config()

  rs.config.enable_device_from_file(config, os.path.join(RECORDING_DIR, recording_path_rs))

  # Configure the pipeline to stream the depth stream
  # Change this parameters according to the recorded bag file resolution
  config.enable_stream(rs.stream.depth, rs.format.z16, 30)

  # Start streaming from file
  pipeline.start(config)
  pipeline.get_active_profile().get_device().as_playback().set_real_time(False)
    
  # Create opencv window to render image in
  cv2.namedWindow("Depth Stream", cv2.WINDOW_AUTOSIZE)
  
  # Create colorizer object
  cv_depth_frames = []
  cv_color_frames = []
  
  # Streaming loop
  for i in tqdm.tqdm(range(num_frames)):

    # Get frameset of depth
    frames = pipeline.wait_for_frames()

    # Get depth frame
    cv_depth_frames.append(frames.get_depth_frame())
    cv_color_frames.append(frames.get_color_frame())
    
  return cv_depth_frames, cv_color_frames

In [28]:
cv_depth_videos = []
cv_color_videos = []

for (i, recording_path_rs) in enumerate(recording_paths_rs):
  print(f'Loading recording {i+1} of {len(recording_paths_rs)} for recording {recording_jsons[i]["Name"]}')
  depth, color = load_recording_rs(recording_path_rs, recording_jsons[i]['Frames'])
  cv_depth_videos.append(depth)
  cv_color_videos.append(color)
  

Loading recording 1 of 44 for recording Session 2023-02-08T22:21:33


100%|██████████| 602/602 [00:03<00:00, 187.66it/s]


Loading recording 2 of 44 for recording Session 2023-02-08T22:22:59


100%|██████████| 600/600 [00:03<00:00, 186.45it/s]


Loading recording 3 of 44 for recording Session 2023-02-08T22:23:46


...

100%|██████████| 151/151 [00:00<00:00, 210.31it/s]


Loading recording 43 of 44 for recording Session 2023-02-13T10:46:44


100%|██████████| 151/151 [00:00<00:00, 207.17it/s]


Loading recording 44 of 44 for recording Session 2023-02-13T10:47:04


100%|██████████| 151/151 [00:00<00:00, 183.96it/s]
